In [1]:
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
df = {}
keys = ['train', 'test']
for k in keys :
    df[k] = pd.read_csv('ag_news/' + k + '.csv', header=None)
    df[k] = df[k][df[k][0].isin([1, 3])]

In [3]:
from vectorizer import cleaner

In [4]:
for k in keys :
    texts = list(df[k][2])
    for i in tqdm_notebook(range(len(texts))) :
        texts[i] = cleaner(texts[i])
    df[k]['text'] = texts

In [5]:
import vectorizer
vec = vectorizer.Vectorizer(min_df=5)

In [6]:
vec.fit(list(df['train']['text']))

In [7]:
vec.seq_text = {}
vec.label = {}
for k in keys :
    vec.seq_text[k] = vec.get_seq_for_docs(list(df[k]['text']))
    vec.label[k] = [1 if (x == 3) else 0 for x in list(df[k][0])]

In [8]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
vectors = Vectors('glove.840B.300d.txt', url=url)

In [9]:
vec.word_dim = vectors.dim

In [10]:
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))

In [11]:
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [12]:
import pickle
pickle.dump(vec, open('ag_news/vec.p', 'wb'))